In [1]:
import cv2
import os
import sys
from datetime import datetime
from ultralytics import YOLO
import matplotlib.pyplot as plt
import duckdb


cwd = os.getcwd()
delimiter = "\\" if "\\" in cwd else "/"
repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("dataImport")]) + delimiter
sys.path.append(repoPath + "dataImport/")
import vidUtils
from vidUtils import getFrame, getCap
import rwWorkingTSDf
from rwWorkingTSDf import readWorkingTSDF, writeWorkingTSDf, getWorkingTSDfPath

In [2]:
model = YOLO("yolo11x-pose.pt")

In [3]:
deviceDescriptor = ["abhik", "bedroompi", "abhik", "piCam-raspberryPi5-Camv3120", "mp4", "piVidCap"]
outputDescriptor = ["abhik", "bedroompi", "ultralytics", "piCam-raspberryPi5-Camv3120", "17pose", "yolo11x-pose"]

In [4]:
vidDf = readWorkingTSDF(deviceDescriptor)
import json

newDfRows = []
for i in range(100):
    row = vidDf.iloc[i]
    ts = row.name
    videoIndex = row['videoIndex']
    videoStartTime = row['videoStartTime']
    videoEndTime = row['videoEndTime']
    frame = getFrame(deviceDescriptor, videoStartTime, videoEndTime, videoIndex)
    result = model(frame, verbose=False)
    toSave = json.dumps({'xyn': result[0].keypoints.xyn.tolist(), 'conf': result[0].keypoints.conf.tolist()})
    newDfRows.append([ts, toSave])


read in 30466 rows from 1 files, retruning 30466 rows


In [21]:
row = vidDf.iloc[0]
ts = row.name
videoIndex = row['videoIndex']
videoStartTime = row['videoStartTime']
videoEndTime = row['videoEndTime']
frame = getFrame(deviceDescriptor, videoStartTime, videoEndTime, videoIndex)
result = model(frame, verbose=False)

In [25]:
result[0].keypoints.xyn.tolist()[0]

[[0.548828661441803, 0.5339609980583191],
 [0.5560177564620972, 0.5020471215248108],
 [0.5317835211753845, 0.510195791721344],
 [0.0, 0.0],
 [0.49121537804603577, 0.4943516254425049],
 [0.5648428797721863, 0.5330049991607666],
 [0.4446115493774414, 0.5631726980209351],
 [0.6148164868354797, 0.611517608165741],
 [0.4482085406780243, 0.7690936923027039],
 [0.6140410304069519, 0.7285906076431274],
 [0.5214909911155701, 0.8189781308174133],
 [0.5419003367424011, 0.7451704144477844],
 [0.4563613533973694, 0.7597834467887878],
 [0.6035729646682739, 0.7961767315864563],
 [0.3784731924533844, 0.7690061330795288],
 [0.5005213022232056, 0.8904839754104614],
 [0.48805490136146545, 0.8636950254440308]]

In [5]:
import pandas as pd
newDf = pd.DataFrame(data=newDfRows, columns=['sampleDT', 'pose'])
newDf = newDf.set_index('sampleDT')
newDf

,pose
sampleDT,
2025-03-13 23:03:00.000596+00:00,"{""xyn"": [[[0.548828661441803, 0.53396099805831..."
2025-03-13 23:03:00.100148+00:00,"{""xyn"": [[[0.5484716296195984, 0.5321729779243..."
2025-03-13 23:03:00.201352+00:00,"{""xyn"": [[[0.5490943193435669, 0.5313501954078..."
2025-03-13 23:03:00.303282+00:00,"{""xyn"": [[[0.5493457913398743, 0.5299937725067..."
2025-03-13 23:03:00.402564+00:00,"{""xyn"": [[[0.5500948429107666, 0.5293601155281..."
...,...
2025-03-13 23:03:09.500148+00:00,"{""xyn"": [[[0.5534456372261047, 0.5194787979125..."
2025-03-13 23:03:09.600136+00:00,"{""xyn"": [[[0.5525461435317993, 0.5217380523681..."
2025-03-13 23:03:09.700147+00:00,"{""xyn"": [[[0.5516005158424377, 0.5251188278198..."


In [6]:
writeWorkingTSDf(outputDescriptor, newDf)

hashes match for 2025-03-13T230300,000596+0000_2025-03-13T230309,900155+0000_821a0a3a_.parquet.gzip


In [7]:
# "SELECT * FROM 'data/video_*.parquet.gzip'"
#maybe I shuold use snappy?

In [ ]:

# Example: Extracting timestamps from a DataFrame index
sTS = newDf.iloc[3].name
eTS = newDf.iloc[97].name

# Open a DuckDB connection
con = duckdb.connect()

# Query execution using parameterized input
query = f"""
SELECT * FROM read_parquet('{wdp}*')
WHERE sampleDT BETWEEN ? AND ?
"""

result = con.execute(query, [sTS, eTS]).df().set_index('sampleDT')
print(sTS)
print(eTS)
print(result)

2025-03-13 23:03:00.303282+00:00
2025-03-13 23:03:09.700147+00:00
                                                                               pose
sampleDT                                                                           
2025-03-13 16:03:00.303282-07:00  {"xyn": [[[0.5493457913398743, 0.5299937725067...
2025-03-13 16:03:00.402564-07:00  {"xyn": [[[0.5500948429107666, 0.5293601155281...
2025-03-13 16:03:00.500204-07:00  {"xyn": [[[0.551284670829773, 0.52800923585891...
2025-03-13 16:03:00.600374-07:00  {"xyn": [[[0.5559868216514587, 0.5183365941047...
2025-03-13 16:03:00.700461-07:00  {"xyn": [[[0.562384307384491, 0.50334841012954...
...                                                                             ...
2025-03-13 16:03:09.300135-07:00  {"xyn": [[[0.5577629208564758, 0.5098487734794...
2025-03-13 16:03:09.400139-07:00  {"xyn": [[[0.5545237064361572, 0.5173035264015...
2025-03-13 16:03:09.500148-07:00  {"xyn": [[[0.5534456372261047, 0.5194787979125...
2025-03-13

In [32]:

# Example: Extracting timestamps from a DataFrame index
sTS = newDf.iloc[3].name.isoformat()
eTS = newDf.iloc[97].name.isoformat()

# Open a DuckDB connection
con = duckdb.connect()

# Query execution using parameterized input
query = f"""
SELECT * FROM read_parquet('{wdp}*')
WHERE sampleDT BETWEEN '{sTS}' AND '{eTS}'
"""

result = con.execute(query).df().set_index('sampleDT')
print(sTS)
print(eTS)
print(result)

2025-03-13T23:03:00.303282+00:00
2025-03-13T23:03:09.700147+00:00
                                                                               pose
sampleDT                                                                           
2025-03-13 16:03:00.303282-07:00  {"xyn": [[[0.5493457913398743, 0.5299937725067...
2025-03-13 16:03:00.402564-07:00  {"xyn": [[[0.5500948429107666, 0.5293601155281...
2025-03-13 16:03:00.500204-07:00  {"xyn": [[[0.551284670829773, 0.52800923585891...
2025-03-13 16:03:00.600374-07:00  {"xyn": [[[0.5559868216514587, 0.5183365941047...
2025-03-13 16:03:00.700461-07:00  {"xyn": [[[0.562384307384491, 0.50334841012954...
...                                                                             ...
2025-03-13 16:03:09.300135-07:00  {"xyn": [[[0.5577629208564758, 0.5098487734794...
2025-03-13 16:03:09.400139-07:00  {"xyn": [[[0.5545237064361572, 0.5173035264015...
2025-03-13 16:03:09.500148-07:00  {"xyn": [[[0.5534456372261047, 0.5194787979125...
2025-03-13

In [39]:
result.iloc[0]['pose']

'{"xyn": [[[0.5493457913398743, 0.5299937725067139], [0.5564596056938171, 0.49916407465934753], [0.5327727198600769, 0.507202684879303], [0.0, 0.0], [0.49235010147094727, 0.49065545201301575], [0.5644938349723816, 0.531312882900238], [0.44555139541625977, 0.5622283220291138], [0.6149812936782837, 0.6118618249893188], [0.44849061965942383, 0.767432451248169], [0.6145949959754944, 0.7294580340385437], [0.5215438604354858, 0.8196727633476257], [0.5393237471580505, 0.7405465841293335], [0.4549259841442108, 0.7557634115219116], [0.6017857789993286, 0.7936225533485413], [0.37912461161613464, 0.7722176909446716], [0.4971955716609955, 0.8887398838996887], [0.4855356216430664, 0.8622958064079285]]], "conf": [[0.9974155426025391, 0.9061453342437744, 0.9973766803741455, 0.13243576884269714, 0.9925325512886047, 0.996394693851471, 0.998034656047821, 0.9773266911506653, 0.981618344783783, 0.9947291016578674, 0.9945664405822754, 0.9592680335044861, 0.9588340520858765, 0.9528013467788696, 0.9330042004

In [41]:
j = json.loads(result.iloc[0]['pose'])

In [45]:
j['xyn'][0][0][0] < 0.55

True

In [ ]:
#jsonb will be good for now we'll use protobuff later once things stablize, and if needed

In [ ]:
# Example: Extracting timestamps from a DataFrame index
sTS = newDf.iloc[3].name.isoformat()
eTS = newDf.iloc[97].name.isoformat()

# Open a DuckDB connection
con = duckdb.connect()

# Query execution using parameterized input
query = f"""
SELECT * FROM read_parquet('{wdp}*')
WHERE sampleDT BETWEEN '{sTS}' AND '{eTS}'
"""

query = f"""
SELECT sampleDT, pose::json -> 'xyn' 
FROM read_parquet('{wdp}*')
WHERE pose::json -> 'xyn' IS NOT NULL
"""

result = con.execute(query).df().set_index('sampleDT')
print(sTS)
print(eTS)
print(result)